In [1]:
import geopandas as gpd
import numpy as np 
import matplotlib.pyplot as plt
import pandas as pd
from shapely import wkt

In [2]:
df = pd.read_csv('ads24-roads.csv', low_memory=False)
# df = df.head(100000)
df

,id,name,road_class,lanes,width,oneway,nodes,wkt
0,4911883,9 Maja,secondary,NaN,NaN,yes,"{1706340509,11956119625,11956119626,1195611962...",LINESTRING(19.143442800000003 51.5878166000000...
1,4911884,NaN,secondary,2,NaN,NaN,"{32127316,4703618827,538482267}","LINESTRING(19.1776191 51.601251700000006,19.17..."
2,4911885,NaN,secondary,2,NaN,NaN,"{31941772,32127316}","LINESTRING(19.1781495 51.601463700000004,19.17..."
3,4918113,NaN,tertiary_link,1,NaN,yes,"{32216651,300116154,10294036067,10294036070,10...","LINESTRING(21.036622400000002 52.1637064,21.03..."
4,4918117,NaN,tertiary,1,NaN,yes,"{32216708,300116160,3391231161,10293604891,300...","LINESTRING(21.0365343 52.162990300000004,21.03..."
...,...,...,...,...,...,...,...,...
2588729,4911754,Aleja Komisji Edukacji Narodowej,tertiary,2,NaN,yes,"{2147593948,2524756147,9492632368,1278816168,9...","LINESTRING(21.0552832 52.142073100000005,21.05..."
2588730,4911759,Akacjowa,residential,NaN,NaN,NaN,"{32126043,10160391828,2511487065,1332421940,32...","LINESTRING(21.0012435 52.2062766,21.0012566 52..."
2588731,4911760,Asfaltowa,residential,1,NaN,yes,"{32126045,1317127866,1317127859,10905645887,26...","LINESTRING(21.006864500000002 52.2072863,21.00..."
2588732,4911761,Opoczyńska,residential,1,NaN,yes,"{32126047,2636842632,10905645879,9420726342,10...",LINESTRING(21.006754700000002 52.2046675000000...


In [3]:
df['nodes'] = df['nodes'].apply(lambda x: list(eval(x)))

df['geometry'] = df['wkt'].apply(wkt.loads)

df['coordinates'] = df['geometry'].apply(lambda geom: list(geom.coords))

def match_ids_to_values(ids, values):
    unique_ids = iter(ids)
    value_to_id = {}

    for value in values:
        if value not in value_to_id:
            value_to_id[value] = next(unique_ids)

    return value_to_id

def close_nodes_to_coordinates(row):
    if len(row['nodes']) == len(row['coordinates']):
        return row
    if row['coordinates'][0] == row['coordinates'][-1]:  #
        row['nodes'].append(row['nodes'][0])
    if len(row['nodes']) != len(row['coordinates']):
        result = match_ids_to_values(row['nodes'], row['coordinates'])
        row['nodes'] = [result[coordinate] for coordinate in row['coordinates']]
    return row
print('checking')
df = df.apply(close_nodes_to_coordinates, axis=1)
print('finished')
print('centroids')

gdf = gpd.GeoDataFrame(df, crs="EPSG:4326", geometry='geometry')
gdf['long'] = gdf.centroid.x
gdf['lat'] = gdf.centroid.y
print('finished')

df_roads = gdf.drop(['wkt', 'geometry', 'coordinates'], axis=1)
df_roads

checking
finished
centroids


/tmp/ipykernel_117813/1861043594.py:32: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  gdf['long'] = gdf.centroid.x
/tmp/ipykernel_117813/1861043594.py:33: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  gdf['lat'] = gdf.centroid.y


finished


,id,name,road_class,lanes,width,oneway,nodes,long,lat
0,4911883,9 Maja,secondary,NaN,NaN,yes,"[11690955571, 11956119625, 11956119626, 119561...",19.143590,51.587732
1,4911884,NaN,secondary,2,NaN,NaN,"[4703618827, 538482267, 32127316]",19.177001,51.601004
2,4911885,NaN,secondary,2,NaN,NaN,"[31941772, 32127316]",19.177884,51.601358
3,4918113,NaN,tertiary_link,1,NaN,yes,"[32216658, 10294036067, 10294036068, 102940360...",21.036676,52.163553
4,4918117,NaN,tertiary,1,NaN,yes,"[300116160, 3391231161, 32216708, 300116158, 3...",21.036949,52.163086
...,...,...,...,...,...,...,...,...,...
2588729,4911754,Aleja Komisji Edukacji Narodowej,tertiary,2,NaN,yes,"[9492632368, 9492632369, 2524756147, 127881614...",21.056129,52.141280
2588730,4911759,Akacjowa,residential,NaN,NaN,NaN,"[10160391828, 1332421940, 2511487065, 32126043...",21.001513,52.205703
2588731,4911760,Asfaltowa,residential,1,NaN,yes,"[9556623584, 9556623585, 9556623586, 32126048,...",21.007434,52.206089
2588732,4911761,Opoczyńska,residential,1,NaN,yes,"[9420726342, 10905645895, 2636842632, 10905645...",21.006184,52.205866


In [ ]:
df_nodes = df[['nodes', 'coordinates']]
df_nodes['nodes'] = df_nodes['nodes'].apply(list)
df_nodes = df_nodes.explode(['nodes', 'coordinates'])
print('exploded')
df_nodes = pd.DataFrame({
    'id': df_nodes['nodes'].astype(int), 
    'wkt': df_nodes['coordinates'].apply(lambda coord: f"POINT ({coord[0]} {coord[1]})")
}).reset_index(drop=True)
print('index')

df_nodes['geometry'] = df_nodes['wkt'].apply(wkt.loads)

gdf_nodes = gpd.GeoDataFrame(df_nodes, crs="EPSG:4326", geometry='geometry')
gdf_nodes['long'] = gdf_nodes.geometry.x
gdf_nodes['lat'] = gdf_nodes.geometry.y

gdf_nodes.to_crs(epsg=2180, inplace=True)
gdf_nodes['x'] = gdf_nodes.geometry.x
gdf_nodes['y'] = gdf_nodes.geometry.y
gdf_nodes = gdf_nodes.drop(['geometry', 'wkt'], axis=1)
gdf_nodes.drop_duplicates(subset="id", inplace=True)
gdf_nodes

/tmp/ipykernel_117813/3513554730.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_nodes['nodes'] = df_nodes['nodes'].apply(list)


exploded
index


,id,long,lat,x,y
0,11690955571,19.143443,51.587817,509934.625969,413490.116246
1,11956119625,19.143487,51.587796,509937.705646,413487.776349
2,11956119626,19.143537,51.587770,509941.174243,413484.892425
3,11956119627,19.143578,51.587737,509943.979359,413481.295636
4,1706340508,19.143649,51.587692,509948.948099,413476.302206
...,...,...,...,...,...
17663275,9556706619,21.003473,52.202872,636868.905796,483758.096512
17663276,9556706620,21.003491,52.202892,636870.049028,483760.264332
17663277,9556706621,21.003529,52.202911,636872.625208,483762.527394
17663278,9556706622,21.003580,52.202927,636876.020031,483764.356915


In [5]:
gdf_nodes.drop_duplicates(subset="id", inplace=True)

In [7]:
a

,id,long,lat,x,y
0,11690955571,19.143443,51.587817,509934.625969,413490.116246
1,11956119625,19.143487,51.587796,509937.705646,413487.776349
2,11956119626,19.143537,51.587770,509941.174243,413484.892425
3,11956119627,19.143578,51.587737,509943.979359,413481.295636
4,1706340508,19.143649,51.587692,509948.948099,413476.302206
...,...,...,...,...,...
17663275,9556706619,21.003473,52.202872,636868.905796,483758.096512
17663276,9556706620,21.003491,52.202892,636870.049028,483760.264332
17663277,9556706621,21.003529,52.202911,636872.625208,483762.527394
17663278,9556706622,21.003580,52.202927,636876.020031,483764.356915
